# 최근 3년(2017-11-01~2020-10-31) 일별 IPTV 영화 순위 집계
### 박스오피스 온라인상영관(IPTV) 일별 영화 순위페이지는 GET방식 요청

In [49]:
# 시계열 데이터 생성
from datetime import datetime
import pandas as pd

time1 = datetime(2017, 11, 1)
time2 = datetime(2020, 10, 31)
print(time2-time1) # 1095

# 2017.11.1 ~ 2020.10.31 1096개의 timestamp 생성
timestamp = pd.date_range(start='2017-11-01', end=None, periods=1096, freq='D')

# 필요한 연, 월, 일을 추출하기 위해 timestamp를 period로 변환 
period = timestamp.to_period(freq='D')
period_str = list(map(str, period))
print(period_str[0], period_str[-1])

1095 days, 0:00:00
2017-11-01 2020-10-31


In [50]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import time

def print_text(obj):
    return obj.text

df = pd.DataFrame()
daycount = 0
for date in period_str:
    daycount += 1
    try:
        start = time.time() # 코드 수행 시간 측정

        url = 'http://www.kobis.or.kr/kobis/business/stat/online/onlineDailyBoxRank.do'

        params = {'CSRFToken' : '-owMZzGEdgQcp30RATiClJuOOKJh3agjHGSpL1bWfRk',
              'loadEnd' : '0',
              'searchType' : 'search',
              'sSearchFrom' : date,
              'sSearchTo' : date}

        response = requests.get(url, params=params) # GET 방식 요청

        response.encoding = "utf-8"
        # print(response.text)

        # HTML 문서를 파싱하고 bs4.BeautifulSoup 객체 생성
        soup = BeautifulSoup(response.text, 'html.parser')

        # 구글 크롬 개발자도구에서 selector copy에서 나오는 child 선택자인 nth-child 를 지원하지 않는다.
        # tr:nth(동적크롤링 셀레늄 드라이버 객체가 지원) -> nth-of-type(정적 크롤링 requests 객체가 지원) 으로 바꿔준다.
        rank_list = soup.select('tr > td:nth-of-type(1)')       # 영화 순위
        movie_list = soup.select('.ellip.per90')                # 영화 이름
        country_list = soup.select('tr > td:nth-of-type(6)')    # 제작 국가
        genre_list = soup.select('tr > td:nth-of-type(7)')      # 영화 장르
        director_list = soup.select('tr > td:nth-of-type(8)')   # 영화 감독
        actor_list = soup.find_all(class_='tal')[2:300:3]       # 영화 배우
        cumulative_audience_list = soup.select('.tar')[2:]      # 영화관 누적 관객수

        iptv = {'date' : [date for _ in range(100)],
                'rank' : list(map(print_text, rank_list)),
                'movie' : list(map(print_text, movie_list)),
                'country' : list(map(print_text, country_list)),
                'genre' : list(map(print_text, genre_list)),
                'director' : list(map(print_text, director_list)),
                'actor' : list(map(print_text, actor_list)),
                'cumulative_audience' : list(map(print_text, cumulative_audience_list))}

        new_df = pd.DataFrame(iptv)
        df = pd.concat([df, new_df], axis=0) # axis=0 행 방향 연결 (default)

        print('수집 완료 날짜:', date, '/ 수집 진행률:', (daycount/len(period_str))*100, '% / 코드 수행시간:', time.time()-start)
    except:
        iptv = {'date' : [date for _ in range(100)],
                'rank' : [np.nan for _ in range(100)],
                'movie' : [np.nan for _ in range(100)],
                'country' : [np.nan for _ in range(100)],
                'genre' : [np.nan for _ in range(100)],
                'director' : [np.nan for _ in range(100)],
                'actor' : [np.nan for _ in range(100)],
                'cumulative_audience' : [np.nan for _ in range(100)]}
        
        new_df = pd.DataFrame(iptv)
        df = pd.concat([df, new_df], axis=0) # axis=0 행 방향 연결 (default)
        
        print('데이터가 없는 날짜:', date) # 2020-04-26
        continue
        
print(df.info())
print(df.isnull().sum(axis=0))
display(df)

수집 완료 날짜: 2017-11-01 / 수집 진행률: 0.09124087591240876 % / 코드 수행시간: 1.35390043258667
수집 완료 날짜: 2017-11-02 / 수집 진행률: 0.18248175182481752 % / 코드 수행시간: 1.4979631900787354
수집 완료 날짜: 2017-11-03 / 수집 진행률: 0.2737226277372263 % / 코드 수행시간: 1.4977567195892334
수집 완료 날짜: 2017-11-04 / 수집 진행률: 0.36496350364963503 % / 코드 수행시간: 1.316842794418335
수집 완료 날짜: 2017-11-05 / 수집 진행률: 0.45620437956204374 % / 코드 수행시간: 1.2746684551239014
수집 완료 날짜: 2017-11-06 / 수집 진행률: 0.5474452554744526 % / 코드 수행시간: 1.2394607067108154
수집 완료 날짜: 2017-11-07 / 수집 진행률: 0.6386861313868614 % / 코드 수행시간: 1.4710125923156738
수집 완료 날짜: 2017-11-08 / 수집 진행률: 0.7299270072992701 % / 코드 수행시간: 4.586845874786377
수집 완료 날짜: 2017-11-09 / 수집 진행률: 0.8211678832116789 % / 코드 수행시간: 3.7616071701049805
수집 완료 날짜: 2017-11-10 / 수집 진행률: 0.9124087591240875 % / 코드 수행시간: 1.4907727241516113
수집 완료 날짜: 2017-11-11 / 수집 진행률: 1.0036496350364963 % / 코드 수행시간: 1.7302045822143555
수집 완료 날짜: 2017-11-12 / 수집 진행률: 1.094890510948905 % / 코드 수행시간: 1.9822328090667725
수집 완료 날짜: 2017-11

,date,rank,movie,country,genre,director,actor,cumulative_audience
0,2017-11-01,1,킬러의 보디가드 (The Hitman's Bodyguard),미국,액션,패트릭 휴즈,"사무엘 L. 잭슨,라이언 레이놀즈,셀마 헤이엑,게리 올드만","1,722,017"
1,2017-11-01,2,대장 김창수 (MAN OF WILL),한국,드라마,이원태,"조진웅,송승헌,정만식,정진영","381,483"
2,2017-11-01,3,발레리안: 천 개 행성의 도시 (VALERIAN : The City of A Tho...,프랑스,액션,뤽 베송,"데인 드한,카라 델러비인,리한나,클라이브 오웬","511,931"
3,2017-11-01,4,희생부활자 (RV: Resurrected Victims),한국,미스터리,곽경택,"김래원,김해숙,성동일,전혜진,장영남","323,628"
4,2017-11-01,5,청년경찰 (Midnight Runners),한국,액션,김주환,"박서준,강하늘,성동일,박하선","5,653,444"
...,...,...,...,...,...,...,...,...
95,2020-10-31,96,아포칼립스: 죽은 자의 주문 (The Huntress: Rune of the Dead),스웨덴,액션,라스무스 터지티스,,0
96,2020-10-31,97,사채왕 페그 (Buffaloed),미국,범죄,타니아 웩슬러,"조이 도이치,제이 코트니,주디 그리어",0
97,2020-10-31,98,라푼젤 (Tangled),미국,애니메이션,"네이슨 그레노,바이론 하워드","제커리 리바이,맨디 무어","1,016,267"
98,2020-10-31,99,시크릿 가든 (The Secret Garden),영국,드라마,마크 먼든,"딕시 에저릭스,콜린 퍼스,줄리 월터스","12,137"


In [51]:
# DataFrame을 csv파일로 저장
df.to_csv('./iptv(dayrank3years).csv', encoding='UTF-8', index=False)